<a href="https://colab.research.google.com/github/mbukacek/ssi/blob/Student_release/ssi_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import math as math
from matplotlib import pyplot as plt

In [4]:
def init_ped_data(t,x,v):
    # initialize ped data container
    # INPUT: vector of init time, init position and init velocity

    # init dataframe with the first ped
    ped_data = pd.DataFrame({'ped_id': 0,
                             't': [[t[0]]],                         # to be list of recorded time
                             'x': [[x[0]]],                         # to be list of recorded x position
                             'v': [[v[0]]],                         # to be list of recorded x velocity
                             }, index = [0])

    # add peds one by one
    rep = range(len(x)-1)
    for i in rep:
        ped_data_n = pd.DataFrame({'ped_id': i+1,
                                't': [[t[i+1]]],
                                'x': [[x[i+1]]],
                                'v': [[v[i+1]]],
                                }, index = [i+1])
        ped_data = pd.concat([ped_data, ped_data_n])

    return ped_data

In [9]:
def one_ped_step(ped_data, ped_idx, act_t, act_v, const):
    # performs one step of one ped with respect to the new velocity

    ped_data.at[ped_idx, 't'] = ped_data.at[ped_idx, 't'] + [act_t]             # '+' works as append

    ped_data.at[ped_idx, 'v'] = ped_data.at[ped_idx, 'v'] + [act_v]

    new_x = ped_data.x[ped_idx][-1] + act_v*const['dt']             # [-1] reffers to the last value
    ped_data.at[ped_idx, 'x'] = ped_data.at[ped_idx, 'x'] + [new_x]

    return ped_data

In [5]:
def get_free_stages(ped_data, ped_idx, const):
  free_stages = list(range(ped_data.x[ped_idx][-1] + 1, const['road_length']))
  for i in range(len(ped_data.index)):
    if ped_data.x[i][-1] > ped_data.x[ped_idx][-1]:
      free_stages.remove(ped_data.x[i][-1])
  return free_stages

In [6]:
def update_v(ped_data, ped_idx, const):
    # New velocity is calculated here according to the group

    v_last = ped_data.v[ped_idx][-1]

    free_stages = get_free_stages(ped_data, ped_idx, const)

    if v_last < const['v_max']:
      v = v_last + 1
    else:
      v = const['v_max']

    while ped_data.x[ped_idx][-1] + v not in free_stages:
      v -= 1

    return v

In [8]:
#============================================#
#              SCRIPT STARTS HERE            #
#============================================#

#======================#
#     PRELIMINARIES    #
#======================#

# Constants - dictionary
const = {'N_ped': 4,                # numer of peds in the system
         'N_step': 10,              # number of steps
         'dt': 1,                   # diffrential step length [s]
         'v_max': 5,                # optimal velocity (scalar) [m/s]
         'road_length': 50          # length of the road
        }

# Init time, positions and velocities
t = [0, 0, 0, 0]
x = [0, 1, 3, 5]
v = [1, 3, 2, 1]

# Init data containers
ped_data = init_ped_data(t,x,v)

In [ ]:
#======================#
#         MODEL        #
#======================#

# Model loop over time
rep = range(const['N_step'])
for i in rep:

    act_t = (i+1)*const['dt']                                       # i+1 is current itteration as i = 0  was defined in init step

    # model loop over other peds
    rep2 = range(const['N_ped'])
    for j in rep2:

        act_v = update_v(ped_data, j, const)
        ped_data = one_ped_step(ped_data, j, act_t, act_v, const)



#======================#
#     POSTPROCESSING   #
#======================#

# Timespace fundamental diagram
plt.figure()
plt.plot(ped_data.t[0], ped_data.x[0], 'r-', label = 'ped 1')
plt.plot(ped_data.t[1], ped_data.x[1], 'g-', label = 'ped 2')
plt.plot(ped_data.t[2], ped_data.x[2], 'b-', label = 'ped 3')
plt.plot(ped_data.t[3], ped_data.x[3], 'k-', label = 'ped 4')
plt.title('Timespace fundamental diagram')
plt.xlabel(r'$t \,\,\mathrm{[s]}$')
plt.ylabel(r'$x \,\,\, \mathrm{[m]}$')
#plt.xlim(0, 10)
#plt.ylim(0, 120)
plt.legend()
plt.show()